In [21]:
    using LinearAlgebra
    using Kronecker
    using DifferentialEquations
    using SparseArrays
    using BenchmarkTools
    using CSV
    using DataFrames

In [2]:
include("../lib/variables.jl")
include("../lib/diffEqSolver.jl")
include("../lib/hamiltonians.jl")
include("../lib/magnetisations.jl")
include("../lib/maths.jl")
include("../lib/operators.jl")
include("../lib/spectralDensity.jl")
include("../lib/states.jl")
include("../lib/superoperators.jl")
include("../lib/transitions.jl")

transitions (generic function with 1 method)

In [3]:
prm = LorPrm3D(2.0, 0.001, 10.0, 2.0, 0.001, 10.0, 2.0, 0.001, 10.0) # Lorentzian parameters
# ang =  CouplAng1D(π/2, 0.0) # Coupling angles
# ang =  CouplAng2D(π/2, 0.0, 0.0, 0.0) # Coupling angles
ang =  CouplAng3D(π/2, 0.0, π/2, π/2, 0.0, 0.0) # Coupling angles
# n = Lev1D(5) # Number of RC levels
# n = Lev2D(10, 10) # Number of RC levels
n = Lev3D(2, 2, 2) # Number of RC levels

Lev3D(2, 2, 2)

In [8]:
T = 0.1

0.01

In [9]:
superop = 𝒮(prm, ang, n, T);

In [7]:
state_init = sparse(ρ0(prm, n, T));

In [10]:
dstate(dρ, ρ, v, t) = mul!(dρ, superop, ρ) # Solves the DE

dstate (generic function with 1 method)

In [11]:
ti, tf, dt = [5000 10000 100]
tspan = (ti, tf)
t = ti:dt:tf

5000:100:10000

In [12]:
prob = ODEProblem(dstate, vec(state_init), tspan)

ODEProblem with uType Base.ReshapedArray{ComplexF64, 1, SparseMatrixCSC{ComplexF64, Int64}, Tuple{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64}}} and tType Int64. In-place: true
timespan: (5000, 10000)
u0: 256-element reshape(::SparseMatrixCSC{ComplexF64, Int64}, 256) with eltype ComplexF64:
      0.8535533905932737 + 0.0im
                     0.0 + 0.0im
                     0.0 + 0.0im
                     0.0 + 0.0im
                     0.0 + 0.0im
                     0.0 + 0.0im
                     0.0 + 0.0im
                     0.0 + 0.0im
    -0.35355339059327373 - 0.0im
                     0.0 + 0.0im
                         ⋮
 -9.370566877313994e-262 - 0.0im
                     0.0 + 0.0im
                     0.0 + 0.0im
                     0.0 + 0.0im
                     0.0 + 0.0im
                     0.0 + 0.0im
                     0.0 + 0.0im
                     0.0 + 0.0im
  3.881415887707559e-262 + 0.0im

In [13]:
solution = solve(prob, alg)

retcode: Success
Interpolation: specialized 7th order lazy interpolation
t: 25770-element Vector{Float64}:
  5000.0
  5000.000363909502
  5000.004003004519
  5000.0262017415125
  5000.064798901784
  5000.111027313065
  5000.165117642245
  5000.228236353699
  5000.300682157737
  5000.381642625505
     ⋮
  9998.081063375039
  9998.333944385698
  9998.586825436552
  9998.839706443861
  9999.092587357385
  9999.345468260859
  9999.598349238024
  9999.85123027213
 10000.0
u: 25770-element Vector{SparseVector{ComplexF64, Int64}}:
   [1  ]  =  0.853553+0.0im
  [9  ]  =  -0.353553-0.0im
  [18 ]  =  1.18123e-87+0.0im
  [26 ]  =  -4.89281e-88-0.0im
  [35 ]  =  1.18123e-87+0.0im
  [43 ]  =  -4.89281e-88-0.0im
  [52 ]  =  1.6347e-174+0.0im
         ⋮
  [197]  =  -4.89281e-88-0.0im
  [205]  =  2.02667e-88+0.0im
  [214]  =  -6.77115e-175-0.0im
  [222]  =  2.8047e-175+0.0im
  [231]  =  -6.77115e-175-0.0im
  [239]  =  2.8047e-175+0.0im
  [248]  =  -9.37057e-262-0.0im
  [256]  =  3.88142e-262+0.0im
   

In [14]:
ρ(t) = reshape(solution(t), (hspace_size(n), hspace_size(n))) # Reformats vector into a density matrix

ρ (generic function with 1 method)

In [17]:
sz_list = [realIfClose(tr(ρ(i)*kronecker(σz, 𝕀(Int(hspace_size(n)/2))))) for i in t]

51-element Vector{Float64}:
 0.7071067811865475
 0.26172242765830794
 0.7781844777542773
 0.4153288464050018
 0.6663837277103581
 0.46479978452471893
 0.6745890379050868
 0.5928698546104286
 0.6770378034626666
 0.6357055285033958
 ⋮
 0.7704345200526672
 0.7705619673440648
 0.7706732483320008
 0.7707694995637876
 0.7708538680044648
 0.7709257498305199
 0.7709878141124261
 0.7710420871950802
 0.7710890204151523

In [23]:
df = DataFrame(hcat(t, sz_list), :auto)
CSV.write("/Users/charliehogg/Desktop/3d_prma_001.csv",  df, header = ["t", "sz"])

"/Users/charliehogg/Desktop/3d_prma_001.csv"